# RMI Detail Transactions Project

## 1.  Load Transactional Data, SalesForce Data, Mavenlink Data, load packages, and convert to Dataframes

I am using three reports from Netsuite to populate this data:  Lists/Employees --> Run report without filters all set to "All".

Search in Netsuite search bar usings "sea" for Adaptive Transaction Search - P&L - Jeremy

Search in Netsuite search bar using "sea" for Mavenlink Time Audit

In Salesforce, use Ginny's link, but filter for "ALL" in Opportunity Record Type.  Also include Revenue Date:  greater or equal to LAST FISCAL YEAR.

![title](img/picture.jpg)

In [1]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from openpyxl.styles.borders import Border, Side


In [2]:
dfHistoric = pd.read_excel('../Data/raw/FY19&FY20_Detail_Trans.xlsx')

In [3]:
from xml.sax import ContentHandler, parse

# Reference https://goo.gl/KaOBG3
class ExcelHandler(ContentHandler):
    def __init__(self):
        self.chars = [  ]
        self.cells = [  ]
        self.rows = [  ]
        self.tables = [  ]
    def characters(self, content):
        self.chars.append(content)
    def startElement(self, name, atts):
        if name=="Cell":
            self.chars = [  ]
        elif name=="Row":
            self.cells=[  ]
        elif name=="Table":
            self.rows = [  ]
    def endElement(self, name):
        if name=="Cell":
            self.cells.append(''.join(self.chars))
        elif name=="Row":
            self.rows.append(self.cells)
        elif name=="Table":
            self.tables.append(self.rows)

excelHandler = ExcelHandler()
parse('../Data/raw/ADAPTIVETransactionSearchPLJeremyResults17.xls', excelHandler)
df1 = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])



In [4]:
excelHandler = ExcelHandler()
parse('../Data/raw/MavenlinkTimeAuditResults375.xls', excelHandler)
dfMavenlink = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])

In [5]:
excelHandler = ExcelHandler()
parse('../Data/raw/Employees588.xls', excelHandler)
employee_list = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])


In [6]:
dfSalesforce = pd.read_html('../Data/raw/report1617115813076.xls')
dfSalesforce = dfSalesforce[0]

In [7]:
dfSalesforceFFS = pd.read_html('../Data/raw/report1617115880508.xls')
dfSalesforceFFS = dfSalesforceFFS[0]

In [8]:
#To get the FFS Balances and Whitney's data from Netsuite: RMI Fee for Service Contract Balances

excelHandler = ExcelHandler()
parse('../Data/raw/RMIFeeforServiceContractBalancesResults539.xls', excelHandler)
dfFFSData = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])

In [9]:
#To get the deferred Revenue data for FFS from Netsuite:  RMI Transaction Search - Revenue Summary

excelHandler = ExcelHandler()
parse('../Data/raw/RMITransactionSearchRevenueSummaryResults385.xls', excelHandler)
dfFFSIncludingDeferred = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])

In [10]:
df1 = pd.concat([dfHistoric, df1], ignore_index=True)

## 2. Data Manipulation

In this section I start manipulting these datasets into somethign more digestable for the end user.  The main manipulation I am doing in this seciton is pulling information out of the "Memo" field that users may want to see seperately.

In [11]:
EmployeeNameandID = employee_list[['Name', 'External ID', 'Job Title']]

In [12]:
EmployeeNameandID['External ID'].replace('', np.nan, inplace=True)

c:\users\jeremy wendt\anaconda3\envs\code\detail_trans_project_fy21\notebooks\venv\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [13]:
len(EmployeeNameandID)

509

In [14]:
#EmployeeNameandID.to_excel('originallist.xlsx')

In [15]:
employee_id_list = employee_list['External ID'].tolist()

In [16]:
employee_id_list = list(filter(None, employee_id_list))

In [17]:
print(len(employee_id_list))

465


In [18]:
df1['Memo'] = df1['Memo'].astype('string')

In [19]:
df1.dtypes

Date                               object
Period                             object
GL Account                         object
Document Number                    object
Program                            object
Initiative                         object
Project                            object
Restriction                        object
Grant                              object
Amount                             object
Vendor Name                        object
Employee Name                      object
Memo                               string
Amount (Credit)                    object
Amount (Debit)                     object
Client                             object
Transaction Number                 object
Type                               object
Vendor Rate                        object
Vendor Contract Expiration Date    object
dtype: object

In [20]:
all_ids = '|'.join(employee_id_list)

In [21]:
def return_first(x):
    try:
        y = x[0]
    except:
        y = ""
    return y

In [22]:
df1['External ID'] = df1['Memo'].str.findall(all_ids).apply(return_first)

In [23]:
employee_name_list = employee_list['Name'].tolist()

In [24]:
employee_name_list = list(filter(None, employee_name_list))

In [25]:
all_names = '|'.join(employee_name_list)

In [26]:
all_names = all_names + "|Agnis Li|Anna Ledden|Greg Hales|Rachel Hohensee|Li Ma"

In [27]:
df1['Match ID Names'] = df1['Memo'].str.findall(all_names).apply(return_first)

In [28]:
df1 = pd.merge(df1, EmployeeNameandID, on = 'External ID', how='left')

In [29]:
df1['Match ID Names'].replace('', np.nan, inplace=True)

In [30]:
df1['Name'].replace('', np.nan, inplace=True)

In [31]:
df1['Name'].fillna(df1['Match ID Names'], inplace=True)

In [32]:
df1['Name'].replace('Agnis Li', 'Agnes Li', inplace=True)
df1['Name'].replace('Anna Ledden', 'Anna  Ledden', inplace=True)
df1['Name'].replace('Rachel Hohensee', 'Rachel T Hohensee', inplace=True)

## Mavenlink Data Manipulation

In [33]:
dfMavenlink['Month'] = pd.DatetimeIndex(dfMavenlink['Date']).month
dfMavenlink['Month'] = dfMavenlink['Month'].map({7: 'Jul 2020', 8: "Aug 2020", 9: "Sep 2020", 10: "Oct 2020", 11: "Nov 2020", 12: "Dec 2020", 1: "Jan 2021", 2: "Feb 2021", 3: "Mar 2021", 4: "Apr 2021", 5: "May 2021", 6: "Jun 2021"})

In [34]:
dfMavenlink['Minutes'] =  dfMavenlink['Duration'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]) * 1)
dfMavenlink['Hours'] =  dfMavenlink['Minutes'] / 60

In [35]:
dfMavenlink['Grant'].replace('', np.nan, inplace=True)
dfMavenlink['Initiative'].replace('', np.nan, inplace=True)
dfMavenlink['Employee'].replace('', np.nan, inplace=True)
dfMavenlink['Case/Task/Event'].replace('', np.nan, inplace=True)
dfMavenlink['Billable'].replace('', np.nan, inplace=True)

In [36]:
dfMavenlink['Grant'].fillna("No Grant", inplace=True)
dfMavenlink['Initiative'].fillna("na", inplace=True)
dfMavenlink['Employee'].fillna("na", inplace=True)
dfMavenlink['Case/Task/Event'].fillna("na", inplace=True)
dfMavenlink['Billable'].fillna("Billable", inplace=True)

In [37]:
dfMavenlink['Employee Hours'] = dfMavenlink.groupby(['Employee','Month'])['Hours'].transform('sum')

In [38]:
dfMavenlink['Percent of Total Monthly Hours'] = dfMavenlink['Hours'] / dfMavenlink['Employee Hours']

In [40]:
#dfMavenlink.to_excel('Output_Mavenlink.xlsx')

## Salesforce Data Manipulation

In [41]:
dfSalesforce['Revenue Status'].replace('', np.nan, inplace=True)
dfSalesforce['Initiative'].replace('', np.nan, inplace=True)
dfSalesforce['Revenue Amount'].replace('', np.nan, inplace=True)
dfSalesforce['RMI Grant Name'].replace('', np.nan, inplace=True)
dfSalesforce['Grant Term - Start Date'].replace('', np.nan, inplace=True)
dfSalesforce['Grant Term - End Date'].replace('', np.nan, inplace=True)

In [42]:
dfSalesforce['Revenue Status'].fillna("na", inplace=True)
dfSalesforce['Initiative'].fillna("na", inplace=True)
dfSalesforce['Revenue Amount'].fillna(0, inplace=True)
dfSalesforce['RMI Grant Name'].fillna("na", inplace=True)
dfSalesforce['Grant Term - Start Date'].fillna("na", inplace=True)
dfSalesforce['Grant Term - End Date'].fillna("na", inplace=True)

In [43]:
dfSalesforceFFS['Amount'].replace('', np.nan, inplace=True)
dfSalesforceFFS['Relationship Manager'].replace('', np.nan, inplace=True)
dfSalesforceFFS['Initiative'].replace('', np.nan, inplace=True)

In [44]:
dfSalesforceFFS['Amount'].fillna(0, inplace=True)
dfSalesforceFFS['Relationship Manager'].fillna('na', inplace=True)
dfSalesforceFFS['Initiative'].fillna('na', inplace=True)

In [45]:
#dfSalesforce.to_excel('Output_Salesforce.xlsx')

## Extract Time Allocation From Memo

In [46]:
allocations = ['BONUS ALLOCATION', 'ADMIN ALLOCATION', 'PTO/HOLIDAY ALLOCATION']

In [47]:
all_allocations = '|'.join(allocations)

In [48]:
df1['ALLOCATION'] = df1['Memo'].str.findall(all_allocations).apply(return_first)

## Extract Project Fields for Heather's report

In [49]:
df1['Project Allocations'] = df1['Project'].str.findall(r'\[(.*?)\]').apply(return_first)

## Add Rollup Accounts

In [50]:
lookups = pd.read_excel('../Data/raw/lookups.xlsx')

In [51]:
lookups.head()

,GL Account,Rollup Account
0,5010 Contributions Revenue : Foundation Contri...,5000 Contributions Revenue
1,5015 Contributions Revenue : Campaign Foundati...,5000 Contributions Revenue
2,5020 Contributions Revenue : Individual Contri...,5000 Contributions Revenue
3,5025 Contributions Revenue : Campaign Corporat...,5000 Contributions Revenue
4,5030 Contributions Revenue : Endowment Contrib...,5000 Contributions Revenue


In [52]:
df1 = pd.merge(df1, lookups, on = 'GL Account', how='left')

In [53]:
df1['Account Digit'] = df1['GL Account'].astype('string').str[0]

In [54]:
df1.columns

Index(['Date', 'Period', 'GL Account', 'Document Number', 'Program',
       'Initiative', 'Project', 'Restriction', 'Grant', 'Amount',
       'Vendor Name', 'Employee Name', 'Memo', 'Amount (Credit)',
       'Amount (Debit)', 'Client', 'Transaction Number', 'Type', 'Vendor Rate',
       'Vendor Contract Expiration Date', 'External ID', 'Match ID Names',
       'Name', 'Job Title', 'ALLOCATION', 'Project Allocations',
       'Rollup Account', 'Account Digit'],
      dtype='object')

In [55]:
df1['Amount'] = df1['Amount'].astype(float)
df1['Amount (Credit)'].replace('', 0, inplace=True)
df1['Amount (Debit)'].replace('', 0, inplace=True)
df1['Amount (Credit)'] = df1['Amount (Credit)'].astype(float)
df1['Amount (Debit)'] = df1['Amount (Debit)'].astype(float)

In [56]:
df1.dtypes

Date                                object
Period                              object
GL Account                          object
Document Number                     object
Program                             object
Initiative                          object
Project                             object
Restriction                         object
Grant                               object
Amount                             float64
Vendor Name                         object
Employee Name                       object
Memo                                string
Amount (Credit)                    float64
Amount (Debit)                     float64
Client                              object
Transaction Number                  object
Type                                object
Vendor Rate                         object
Vendor Contract Expiration Date     object
External ID                         object
Match ID Names                      object
Name                                object
Job Title  

## Add Reclass Field and Closeout field

In [57]:
account_exceptions = ['To reclass', 'Reclass']

In [58]:
Accounting_Moves = '|'.join(account_exceptions)

In [59]:
closeout = 'close out'

In [60]:
df1['Accounting Adjustments'] = df1['Memo'].str.findall(Accounting_Moves).apply(return_first)

In [61]:
df1['Accounting Close Out'] = df1['Memo'].str.findall(closeout).apply(return_first)

In [62]:
df1['Vendor Contract Expiration Date'] = pd.to_datetime(df1['Vendor Contract Expiration Date'])

In [63]:
df1['Vendor Contract Expiration Date'] = df1['Vendor Contract Expiration Date'].dt.date

## ADDING TITLES INSTEAD OF NAMES

In [64]:
EmployeeNameandID['Job Title'].value_counts()

Senior Associate                  72
Associate                         69
Manager                           66
Intern                            47
Principal                         39
                                  ..
BRC Manager                        1
Wheel Runner                       1
Development Marketing Director     1
Program Services Manager           1
Email Marketing Lead               1
Name: Job Title, Length: 112, dtype: int64

In [65]:
uniquetitles = EmployeeNameandID['Job Title'].unique()

In [66]:
Titleslist = uniquetitles.tolist()

In [67]:
def createEmployeeTitlesdf(title, df):
    newdf = df[df['Job Title']==title]
    newdf.reset_index(drop=True, inplace=True)
    newdf['PositionID'] = newdf['Job Title'] + newdf.index.astype(str)    
    return newdf.to_dict()

In [68]:
DictList = {}

for i in range(len(Titleslist)):
    newdict = createEmployeeTitlesdf(Titleslist[i], EmployeeNameandID)
    DictList[i] = newdict

<ipython-input-67-933c5a1db0ab>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['PositionID'] = newdf['Job Title'] + newdf.index.astype(str)


In [69]:
dflist = []

for i in range(len(DictList)):
    df = pd.DataFrame(DictList[i])
    dflist.append(df)

In [70]:
finalemployeedf = pd.concat(dflist).reset_index(drop=True)

In [71]:
employeelisttomerge = finalemployeedf[['Name', 'PositionID']]

In [72]:
#employeelisttomerge.to_excel('listwithposition.xlsx')

In [73]:
len(employeelisttomerge)

509

In [74]:
df1 = pd.merge(df1, employeelisttomerge, on = 'Name', how='left')

In [75]:
df1['Initiative'].replace('', np.nan, inplace=True)
df1['Project'].replace('', np.nan, inplace=True)
df1['Grant'].replace('', np.nan, inplace=True)
df1['GL Account'].replace('', np.nan, inplace=True)
df1['Memo'].replace('', np.nan, inplace=True)
df1['Transaction Number'].replace('', np.nan, inplace=True)
df1['Vendor Name'].replace('', np.nan, inplace=True)
df1['Vendor Rate'].replace('', np.nan, inplace=True)
df1['Vendor Contract Expiration Date'].replace('', np.nan, inplace=True)
df1['Program'].replace('', np.nan, inplace=True)
df1['Project Allocations'].replace('', np.nan, inplace=True)
df1['Project Allocations'].replace('e', np.nan, inplace=True)
df1['Accounting Adjustments'].replace('To reclass', 'Reclass', inplace=True)
df1['Client'].replace('', np.nan, inplace=True)
df1['Rollup Account'].replace('', np.nan, inplace=True)

c:\users\jeremy wendt\anaconda3\envs\code\detail_trans_project_fy21\notebooks\venv\lib\site-packages\pandas\core\missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x


In [76]:
df1['Initiative'].fillna("na", inplace=True)
df1['Project'].fillna("na", inplace=True)
df1['Grant'].fillna("No Grant", inplace=True)
df1['GL Account'].fillna("na", inplace=True)
df1['Memo'].fillna("na", inplace=True)
df1['Transaction Number'].fillna("na", inplace=True)
df1['Vendor Name'].fillna("na", inplace=True)
df1['Vendor Rate'].fillna("na", inplace=True)
df1['Vendor Contract Expiration Date'].fillna("na", inplace=True)
df1['Program'].fillna("na", inplace=True)
df1['Project Allocations'].fillna("Other", inplace=True)
df1['Client'].fillna("na", inplace=True)
df1['Rollup Account'].replace("New Account", np.nan, inplace=True)

## Add Further Heather Calcs

In [77]:
df1['Inst or Reclass'] = df1['Account Digit'] + df1['Accounting Adjustments']

In [78]:
df1['Inst or Reclass'].value_counts()

7           253197
8            44622
5            22240
6             8078
7Reclass      4057
8Reclass       950
5Reclass       141
6Reclass         4
9                2
Name: Inst or Reclass, dtype: Int64

In [79]:
df1['Inst or Reclass'] = df1['Inst or Reclass'].map({'7': "Expense", '8': "Institutional Overhead", '5': "Revenue", '6': "Revenue", '7Reclass': "Expense Reclass", '8Reclass': "Inst Overhead Reclass", '5Reclass': "Revenue Reclass", '6Reclass': "Revenue Reclass"})

## Unrestricted Columns Added

In [80]:
def return_grant(x):
    if x != "No Grant":
        return "Yes"
    else: 
        return "No"
    
    

In [81]:
df1['grant?'] = df1['Grant'].apply(return_grant)

In [82]:
df1["Restricted Revenue"] = df1['grant?'] + df1['Account Digit']

In [83]:
df1["Restricted Revenue"] = df1["Restricted Revenue"].map({'No7': "No", 'Yes7': "Restricted Revenue", 'No8': "No", 'No5': "No", 'Yes8': "Restricted Revenue", 'Yes6': "No", 'No6': "No", 'Yes5': "No"})

In [84]:
df1['Restricted Revenue'] = np.where(df1['Restricted Revenue'] == "Restricted Revenue", df1['Amount'], 0)

In [85]:
df1["Rev or Exp"] = df1["Account Digit"].map({'6': 'Revenue', '5': 'Revenue', '7': 'Expense', '8': 'Expense'})

In [86]:
df1["Rev or Exp"].value_counts()

Expense    302826
Revenue     30463
Name: Rev or Exp, dtype: int64

In [87]:
df1['Multilateral'] = np.where(df1['Rev or Exp'] == "Revenue", df1['Amount'], 0)

In [88]:
df1['General Expense'] = np.where(df1['Rev or Exp'] == "Expense", df1['Amount'], 0)

In [89]:
df1['Unrestricted'] = - df1['Restricted Revenue'] - df1['Multilateral'] + df1['General Expense']

In [90]:
df1 = df1.drop('Match ID Names',1)

## Build FFS Tables

In [91]:
df1['Project3YrRev'] = df1.groupby(['Project','Rev or Exp'])['Amount'].transform('sum')

dfrevenuetable = df1[['Project', 'Project3YrRev','Rev or Exp']]

dfrevenuetable = dfrevenuetable[dfrevenuetable['Rev or Exp']=='Revenue']

initiative_program_table = df1[['Program', 'Initiative', 'Project']]

initiative_program_table = initiative_program_table.drop_duplicates()

In [92]:
dfFFSData = dfFFSData.rename(columns={'Name':'Project'})
dfFFSIncludingDeferred = dfFFSIncludingDeferred.rename(columns={'Sum of Total Revenue Recognized':'Total Revenue Recognized'})

In [93]:
dfFFSData = pd.merge(dfFFSData, dfFFSIncludingDeferred, on = 'Project', how='left')


In [94]:
dfFFSData['Total Revenue Recognized'].replace('', np.nan, inplace=True)
dfFFSData['Total Revenue Recognized'].fillna(0, inplace=True)
dfFFSData['Reimbursables NTE'].replace('', np.nan, inplace=True)
dfFFSData['Reimbursables NTE'].fillna(0, inplace=True)
dfFFSData['Contract Amount'].replace('', np.nan, inplace=True)
dfFFSData['Contract Amount'].fillna(0, inplace=True)
dfFFSData['Remaining Contract Balance'].replace('', np.nan, inplace=True)
dfFFSData['Remaining Contract Balance'].fillna(0, inplace=True)

In [95]:
dfFFSData['Contract Amount'] = dfFFSData['Contract Amount'].astype(float)
dfFFSData['Total Revenue Recognized'] = dfFFSData['Total Revenue Recognized'].astype(float)
dfFFSData['Reimbursables NTE'] = dfFFSData['Reimbursables NTE'].astype(float)
dfFFSData['Reimbursables Actual'] = dfFFSData['Reimbursables Actual'].astype(float)

In [96]:
dfFFSData['Remaining Revenue'] = dfFFSData['Contract Amount'] + dfFFSData['Reimbursables NTE'] - dfFFSData['Total Revenue Recognized']

In [97]:
dfFFSData.loc[dfFFSData['Remaining Revenue'] <= 0, 'Remaining Revenue'] = "NA"
dfFFSData.loc[dfFFSData['Contract Amount'] <= 0, 'Contract Amount'] = "NA"

In [98]:
dfFFSData['Contract Start Date'] = pd.to_datetime(dfFFSData['Contract Start Date']).dt.strftime("%m/%d/%Y")
dfFFSData['Contract End Date'] = pd.to_datetime(dfFFSData['Contract End Date']).dt.strftime("%m/%d/%Y")

In [99]:
dfFFSData

,Project,Project Type,Contract?,Contract Start Date,Contract End Date,Invoice Type,Invoicing Notes,Current Month Action,Revenue Recognition Notes,Contract Amount,Project Invoicing Actual,Reimbursables NTE,Reimbursables Actual,Remaining Contract Balance,Total Revenue Recognized,Remaining Revenue
0,Nebraska Conservation Voters (NECV) : Nebraska...,FFS,Yes,12/03/2020,NaN,Fixed fee,Invoice 25% upfront and remainder on completion,"1901 sent to Kristine 2/17, 25%, Kristine sent...",Point in time; Note no revenue has been recogn...,10500,2625.00,0.0,0.00,7875.00,0.000000e+00,10500
1,NREL : NREL - Islands Playbook 2.0 (FFS),FFS,No,NaN,NaN,- None -,"3/5 no contract yet per Shelley, Note first $4...",- None -,- None -,NA,4000.00,0.0,0.00,0,4.000000e+03,NA
2,Breitling SA : Breitling LCA,FFS,Yes,11/16/2020,NaN,Fixed fee plus reimbursables,Invoice in full when work is complete (likely ...,"1896 sent to client 2/9, ready to invoice Feb ...",Point in time,24080,4016.00,0.0,0.00,20064.00,4.016000e+03,20064
3,Connecticut Public Utilities Regulatory Author...,FFS,Yes,12/07/2020,12/31/2021,Hourly plus reimbursables,"Per person - include Phase, task, person name,...","3/5 1939 sent to client, sent 1821 to client 2/9",Over time,200000,10161.75,0.0,0.00,189838.25,1.016175e+04,189838
4,"Apple, Inc. : Apple Supply Chain EE Finance",FFS,No,NaN,NaN,Hourly,"BPO 6000487884 dated 12/14/20, delivery deadli...",- None -,Over time,NA,13985.00,0.0,0.00,0,1.398500e+04,NA
5,European Climate Foundation : GCoM/ECF: Case S...,FFS,Yes,01/01/2021,07/31/2021,Fixed fee,"Contract in EUR; 25,000 EUR converted to USD i...",First invoice generated as an SO prior to cont...,"Point in time; first invoice for 12,500 EUR is...",30374.7,14141.25,0.0,0.00,16233.40,0.000000e+00,30374.7
6,NACFE : Run on Less - Electric,FFS,Yes,01/01/2021,12/31/2021,Hourly plus reimbursables,"Invoice actual monthly expenses, no budget","1955 sent to client 3/10, 1899 sent to Mike/cl...",Over time,NA,15114.38,0.0,0.00,0,1.511438e+04,NA
7,Association for Energy Affordability : REALIZE...,State,Yes,06/30/2020,03/31/2024,Hourly plus reimbursables,"Invoices to record revenue only, see ML/BL/GOV...","1962 in March for Feb work started 3/12, Feb 1...",Over time,172658,15177.76,0.0,0.00,157480.24,1.517776e+04,157480
8,3Degrees : 3D Utility Product Strategy,FFS,Yes,12/04/2020,06/30/2021,Fixed fee,Fixed fee amount when deliverable is met,"1932 sent to client 3/3, 1905 sent to Client 2...",Over time,78116,15500.00,0.0,0.00,62616.00,1.550000e+04,62616
9,"Voloridge Investment Management, LLC : Volorid...",FFS,Yes,06/02/2019,NaN,Hourly,10/20 confirming how/when to invoice- idea is ...,"Not in Feb/Jan (no change from Dec), Not in Oc...",Over time,41000,16000.00,0.0,0.00,25000.00,1.600000e+04,25000


In [100]:
dfFFSDataV2 = dfFFSData

dfFFSDataV2 = pd.merge(dfFFSDataV2, initiative_program_table, on = 'Project', how='left')

dfFFSDataV2 = pd.merge(dfFFSDataV2, dfrevenuetable, on = 'Project', how='left')

dfFFSDataV2 = dfFFSDataV2.drop_duplicates()

In [101]:
dfFFSDataV2 = dfFFSDataV2[dfFFSDataV2['Project'] != "Overall Total"]

In [102]:
dfFFSDataV2['Rev or Exp'].replace('', np.nan, inplace=True)
dfFFSDataV2['Rev or Exp'].fillna(0, inplace=True)
dfFFSDataV2['Reimbursables NTE'].replace('', np.nan, inplace=True)
dfFFSDataV2['Reimbursables NTE'].fillna('NA', inplace=True)
dfFFSDataV2['Contract Start Date'].replace('', np.nan, inplace=True)
dfFFSDataV2['Contract Start Date'].fillna('NA', inplace=True)
dfFFSDataV2['Contract End Date'].replace('', np.nan, inplace=True)
dfFFSDataV2['Contract End Date'].fillna('NA', inplace=True)

In [104]:
dfFFSDataV2.to_excel("FFSOpen.xlsx")

In [105]:
dfFFSDataZeros = dfFFSDataV2[dfFFSDataV2['Rev or Exp']==0]

## Add critical datapoints to Netsuite Report from RMI Fee for Service Report 

In [106]:
dfFFSDataedited = dfFFSData[['Project', 'Project Type', 'Contract?',
       'Contract Start Date', 'Contract End Date', 'Invoice Type',
       'Invoicing Notes', 'Current Month Action', 'Revenue Recognition Notes',
       'Contract Amount', 'Total Revenue Recognized', 'Reimbursables NTE',
       'Reimbursables Actual', 'Remaining Revenue']]


dfFFSDataedited.columns

Index(['Project', 'Project Type', 'Contract?', 'Contract Start Date',
       'Contract End Date', 'Invoice Type', 'Invoicing Notes',
       'Current Month Action', 'Revenue Recognition Notes', 'Contract Amount',
       'Total Revenue Recognized', 'Reimbursables NTE', 'Reimbursables Actual',
       'Remaining Revenue'],
      dtype='object')

In [107]:
df1 = pd.merge(df1, dfFFSDataedited, on = 'Project', how='left')

In [108]:
df1.columns

Index(['Date', 'Period', 'GL Account', 'Document Number', 'Program',
       'Initiative', 'Project', 'Restriction', 'Grant', 'Amount',
       'Vendor Name', 'Employee Name', 'Memo', 'Amount (Credit)',
       'Amount (Debit)', 'Client', 'Transaction Number', 'Type', 'Vendor Rate',
       'Vendor Contract Expiration Date', 'External ID', 'Name', 'Job Title',
       'ALLOCATION', 'Project Allocations', 'Rollup Account', 'Account Digit',
       'Accounting Adjustments', 'Accounting Close Out', 'PositionID',
       'Inst or Reclass', 'grant?', 'Restricted Revenue', 'Rev or Exp',
       'Multilateral', 'General Expense', 'Unrestricted', 'Project3YrRev',
       'Project Type', 'Contract?', 'Contract Start Date', 'Contract End Date',
       'Invoice Type', 'Invoicing Notes', 'Current Month Action',
       'Revenue Recognition Notes', 'Contract Amount',
       'Total Revenue Recognized', 'Reimbursables NTE', 'Reimbursables Actual',
       'Remaining Revenue'],
      dtype='object')

In [109]:
df1['Project Type'].replace('', np.nan, inplace=True)
df1['Contract?'].replace('', np.nan, inplace=True)
df1['Contract Start Date'].replace('', np.nan, inplace=True)
df1['Contract End Date'].replace('', np.nan, inplace=True)
df1['Invoice Type'].replace('', np.nan, inplace=True)
df1['Invoicing Notes'].replace('', np.nan, inplace=True)
df1['Current Month Action'].replace('', np.nan, inplace=True)
df1['Revenue Recognition Notes'].replace('', np.nan, inplace=True)
df1['Contract Amount'].replace('', np.nan, inplace=True)
df1['Total Revenue Recognized'].replace('', np.nan, inplace=True)
df1['Reimbursables NTE'].replace('', np.nan, inplace=True)
df1['Reimbursables Actual'].replace('', inplace=True)
df1['Remaining Revenue'].replace('', np.nan, inplace=True)

In [110]:
df1['Project Type'].fillna("na", inplace=True)
df1['Contract?'].fillna("na", inplace=True)
df1['Contract Start Date'].fillna("na", inplace=True)
df1['Contract End Date'].fillna("na", inplace=True)
df1['Invoice Type'].fillna("na", inplace=True)
df1['Invoicing Notes'].fillna("na", inplace=True)
df1['Current Month Action'].fillna("na", inplace=True)
df1['Revenue Recognition Notes'].fillna("na", inplace=True)
df1['Contract Amount'].fillna("na", inplace=True)
df1['Total Revenue Recognized'].fillna("na", inplace=True)
df1['Reimbursables NTE'].fillna("na", inplace=True)
df1['Reimbursables Actual'].fillna("na", inplace=True)
df1['Remaining Revenue'].fillna("na", inplace=True)

## Output manipulated DF to Excel

In [111]:
#df1.to_excel('Output_Netsuite.xlsx')

# 3.  Data Segmentation by Program

Here I'm taking the manipulated data and ultimately creating a dataframe to filter out each program's restricted revenue.

In [112]:
Programs = list(df1['Program'].unique())

In [113]:
print(Programs)

['Development', 'Carbon-Free Electricity', 'Urban Transformation', 'Building Electrification', 'Operations', 'Beijing', 'China', 'Islands', 'Industry', 'Africa', 'Carbon-Free Buildings', 'Strategic Engagement and Analysis Group', 'Global Climate Finance', 'India', 'Shine', 'Carbon-Free Mobility', 'Emerging Energy Solutions', 'WattTime', 'Business Renewable Center', 'Breakthrough Technology', 'Communications', 'Energy Web Foundation', 'Sustainable Finance', 'na', 'Climate Aligned Industries', 'Climate Intelligence', 'SE Asia', 'US', 'Energy Transition Academy']


In [114]:
def create_program(program):
    firstdf = df1[df1['Program']==program]
    seconddf = firstdf[firstdf['Grant'] != 'No Grant'] 
    thirddf = seconddf[(seconddf['Account Digit']=='7')|(seconddf['Account Digit']=='8')]
    if thirddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return thirddf

In [115]:
Development = create_program('Development')
Islands = create_program('Islands')
India = create_program('India')
Operations = create_program('Operations')
China = create_program('China')
Breakthrough_Technology = create_program('Breakthrough Technology')
CarbonFree_Mobility = create_program('Carbon-Free Mobility')
Climate_Aligned_Industries = create_program('Climate Aligned Industries')
Global_Climate_Finance = create_program('Global Climate Finance')
Strategic_Engagement_and_Analysis_Group = create_program('Strategic Engagement and Analysis Group')
CarbonFree_Buildings = create_program('Carbon-Free Buildings')
WattTime = create_program('WattTime')
CarbonFree_Electricty = create_program('Carbon-Free Electricity')
Urban_Transformation = create_program('Urban Transformation')
Climate_Intelligence = create_program('Climate Intelligence')
Industry = create_program('Industry')
SE_Asia = create_program('SE Asia')
Africa = create_program('Africa')
US = create_program('US')
Building_Electrification = create_program('Building Electrification')
Energy_Transition_Academy = create_program('Energy Transition Academy')
Beijing = create_program('Beijing')

Development DF is EMPTY


In [116]:
Developing_Economies_Rollup = pd.concat([Islands, Africa, SE_Asia, Energy_Transition_Academy], ignore_index = True, sort=False)
CarbonFreeBuildings_Rollup = pd.concat([CarbonFree_Buildings, Building_Electrification], ignore_index = True, sort=False)
China_Rollup = pd.concat([China, Beijing], ignore_index = True, sort=False)

## 4. Create Reports Path and Manipulate File Names

I will need to come up with naming conventions that work over time for this project, so in this step I do just that.   Also I am loading the Openpyxl packages that i'm going to need for the rest of this program.

In [117]:
import os

path = r'C:\Users\Jeremy Wendt\anaconda3\envs\Code\Detail_Trans_Project_FY21\reports\Detail_Trans_Reports'

In [118]:
from datetime import datetime
today = datetime.today()
datem = datetime(today.year, today.month-1, 1)
timestamp = "Detail_Transactions_"+datem.strftime("%b_%Y")

In [119]:
print(timestamp+".xlsx")

Detail_Transactions_Feb_2021.xlsx


In [120]:
from openpyxl.styles import Color, PatternFill, Font, Border
greyFill = PatternFill(start_color='00C0C0C0',
                   end_color='00C0C0C0',
                   fill_type='solid')

## 5.  Create Pivot Table, Add Subtotals, Export to Excel, and Combine All These Functions into One

In [121]:
def create_multi_pivot(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant', 'GL Account', 'Memo', 'Transaction Number', 'Vendor Name', 'Vendor Rate', 'Vendor Contract Expiration Date'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [122]:
def add_subtotal_multi(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', '', '', '', '', 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '', '', '', '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [123]:
def to_excel(pivot, str):
    pivot.to_excel(path+str, freeze_panes = (1,9), engine = 'openpyxl', sheet_name = 'Restricted_Rev_Detail', float_format = "%.2f")

In [124]:


def edit_workbook(file):
    book = load_workbook(file)
    sheet = book.active #active means last opened sheet
    sheet['A1']= "Initiative"
    sheet['B1']= "Project"
    sheet['C1']= "Grant"
    sheet['D1']= "GL Account"
    sheet['E1']= "Memo"
    sheet['F1']= "Transaction Number"
    sheet['G1']= "Vendor Name"
    sheet['H1']= "Vendor Rate"
    sheet['I1']= "Vendor Contract Expiration Date" 
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet['F1'].font = Font(bold=True)
    sheet['G1'].font = Font(bold=True)
    sheet['H1'].font = Font(bold=True)
    sheet['I1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 20
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 12.5
    sheet.column_dimensions['K'].width = 12.5
    sheet.column_dimensions['L'].width = 12.5
    sheet.column_dimensions['M'].width = 12.5
    sheet.column_dimensions['N'].width = 12.5
    sheet.column_dimensions['O'].width = 12.5
    sheet.column_dimensions['P'].width = 12.5
    sheet.column_dimensions['Q'].width = 12.5
    sheet.column_dimensions['R'].width = 12.5
    sheet.column_dimensions['S'].width = 12.5
    sheet.column_dimensions['T'].width = 12.5   
    sheet.column_dimensions['U'].width = 12.5   
    sheet.column_dimensions['V'].width = 12.5   
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(10, 23):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0.00_);[Red]("$"#,##0.00)'
    for col in range(1, 10):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=9).value == "Subtotal":
            for j in range(10,23):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 55
#    sheet.protection.sheet = True
    book.save(file)

## Testing for unmerging cells

In [125]:
# THIS CODE WILL UNMERGE ALL THE CELLS IN A SHEET.   
def unmerge_rows(file):
    for col in range(1,5):
        rowstounmerge = []
        book = load_workbook(file)
        testsheet = book['Restricted_Rev_Detail'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                rowstounmerge.append(i)
        rowstounmerge.append(testsheet.max_row)
        for i in range(1,len(rowstounmerge)-1):        
            try:
                testsheet.unmerge_cells(start_row=rowstounmerge[i], start_column=col, end_row = rowstounmerge[i+1]-1, end_column=col) 
            except:
                pass
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['Restricted_Rev_Detail'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                continue
            else:
                testsheet.cell(row=i, column=col).value = testsheet.cell(row=i-1, column=col).value
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['Restricted_Rev_Detail'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=9).value == "Subtotal":
                testsheet.cell(row=i, column=col).value = None
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['Restricted_Rev_Detail'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=1).value == "Grand":
                testsheet.cell(row=i, column=2).value = None
                testsheet.cell(row=i, column=3).value = None
                testsheet.cell(row=i, column=4).value = None
        book.save(file) 

In [126]:
def format_multi_pivot(Group, str):
    step1 = create_multi_pivot(Group)
    step2 = add_subtotal_multi(step1)
    to_excel(step2, str)
    edit_workbook(path+str)
    unmerge_rows(path+str)

## 6.  Create Restricted Revenue Files

In [127]:
'''
%%time
format_multi_pivot(India, '\\India_'+timestamp+".xlsx")
format_multi_pivot(Operations, '\\Operations_'+timestamp+".xlsx")
format_multi_pivot(China_Rollup, '\\China_'+timestamp+".xlsx")
format_multi_pivot(Breakthrough_Technology, '\\Breakthrough_Technology-'+timestamp+".xlsx")
'''
format_multi_pivot(CarbonFree_Mobility, '\\CarbonFree_Mobility_'+timestamp+".xlsx")
'''
format_multi_pivot(Climate_Aligned_Industries, '\\Climate_Aligned_Industries_'+timestamp+".xlsx")
format_multi_pivot(Strategic_Engagement_and_Analysis_Group, '\\Strategic_Engagement_and_Analysis_Group_'+timestamp+".xlsx")
format_multi_pivot(US, '\\US_'+timestamp+".xlsx")  
format_multi_pivot(Urban_Transformation, '\\Urban_Transformation_'+timestamp+".xlsx")
format_multi_pivot(CarbonFreeBuildings_Rollup, '\\CarbonFree_Buildings_'+timestamp+".xlsx")
format_multi_pivot(WattTime, '\\WattTime_'+timestamp+".xlsx")
format_multi_pivot(CarbonFree_Electricty, '\\CarbonFree_Electricity_'+timestamp+".xlsx")
format_multi_pivot(Climate_Intelligence, '\\Climate_Intelligence_'+timestamp+".xlsx")
format_multi_pivot(Industry, '\\Industry_'+timestamp+".xlsx")
format_multi_pivot(Developing_Economies_Rollup, '\\Developing_Economies_'+timestamp+".xlsx")
format_multi_pivot(Global_Climate_Finance, '\\Global_Climate_Finance_'+timestamp+".xlsx")  
'''

'\nformat_multi_pivot(Climate_Aligned_Industries, \'\\Climate_Aligned_Industries_\'+timestamp+".xlsx")\nformat_multi_pivot(Strategic_Engagement_and_Analysis_Group, \'\\Strategic_Engagement_and_Analysis_Group_\'+timestamp+".xlsx")\nformat_multi_pivot(US, \'\\US_\'+timestamp+".xlsx")  \nformat_multi_pivot(Urban_Transformation, \'\\Urban_Transformation_\'+timestamp+".xlsx")\nformat_multi_pivot(CarbonFreeBuildings_Rollup, \'\\CarbonFree_Buildings_\'+timestamp+".xlsx")\nformat_multi_pivot(WattTime, \'\\WattTime_\'+timestamp+".xlsx")\nformat_multi_pivot(CarbonFree_Electricty, \'\\CarbonFree_Electricity_\'+timestamp+".xlsx")\nformat_multi_pivot(Climate_Intelligence, \'\\Climate_Intelligence_\'+timestamp+".xlsx")\nformat_multi_pivot(Industry, \'\\Industry_\'+timestamp+".xlsx")\nformat_multi_pivot(Developing_Economies_Rollup, \'\\Developing_Economies_\'+timestamp+".xlsx")\nformat_multi_pivot(Global_Climate_Finance, \'\\Global_Climate_Finance_\'+timestamp+".xlsx")  \n'

## 7.  Add ML and Consulting Fee Tab

In [128]:
def create_df_FeeforService(program):
    firstdf = df1[df1['Program']==program]
    seconddf = firstdf[(firstdf['Account Digit']=='5')|(firstdf['Account Digit']=='6')]
    if seconddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return seconddf

In [129]:
DevelopmentFFS = create_df_FeeforService("Development")
IslandsFFS = create_df_FeeforService("Islands")
IndiaFFS = create_df_FeeforService("India")
OperationsFFS = create_df_FeeforService("Operations")
ChinaFFS = create_df_FeeforService("China")
Breakthrough_Technology_FFS = create_df_FeeforService("Breakthrough Technology")
CarbonFree_MobilityFFS = create_df_FeeforService("Carbon-Free Mobility")
ClimateAligned_IndustriesFFS = create_df_FeeforService("Climate Aligned Industries")
CommunicationsFFS = create_df_FeeforService("Communications")
GlobalClimate_FinanceFFS = create_df_FeeforService("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupFFS = create_df_FeeforService("Strategic Engagement and Analysis Group")
Carbon_Free_BuildingsFFS = create_df_FeeforService("Carbon-Free Buildings")
Watttime_FFS = create_df_FeeforService("WattTime")
Carbon_Free_ElectricityFFS = create_df_FeeforService("Carbon-Free Electricity")
UrbanTrans_FFS = create_df_FeeforService("Urban Transformation")
ClimateIntelFFS = create_df_FeeforService("Climate Intelligence")
IndustryFFS = create_df_FeeforService("Industry")
SEAsiaFFS = create_df_FeeforService("SE Asia")
AfricaFFS = create_df_FeeforService("Africa")
USFFS = create_df_FeeforService("US")
BuildingElectrificationFFS = create_df_FeeforService("Building Electrification")
Energy_Transition_AcademyFFS = create_df_FeeforService("Energy Transition Academy")
BeijingFFS = create_df_FeeforService("Beijing")

In [130]:
Developing_Economies_RollupFFS = pd.concat([IslandsFFS, AfricaFFS, SEAsiaFFS, Energy_Transition_AcademyFFS], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupFFS = pd.concat([Carbon_Free_BuildingsFFS, BuildingElectrificationFFS], ignore_index = True, sort=False)
China_RollupFFS = pd.concat([ChinaFFS,BeijingFFS], ignore_index = True, sort=False)


In [131]:
def create_FFS_pivot(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant', 'GL Account', 'Memo', 'Client', 'Transaction Number'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [132]:
def add_subtotal_FFS(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', '', '',  'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '',  '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [133]:
def to_excel_FFS(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="ML & FFS Revenue")
    writer.save()
    writer.close()    

In [134]:
def edit_workbook_FFS(file):
    book = load_workbook(file)
    sheet = book["ML & FFS Revenue"] #active means last opened sheet
    sheet['A1']= "Initiative"
    sheet['B1']= "Project"
    sheet['C1']= "Grant"
    sheet['D1']= "GL Account"
    sheet['E1']= "Memo"
    sheet['F1']= "Client"
    sheet['G1']= "Transaction Number" 
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet['F1'].font = Font(bold=True)
    sheet['G1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 20
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['J'].width = 12.5
    sheet.column_dimensions['K'].width = 12.5
    sheet.column_dimensions['L'].width = 12.5
    sheet.column_dimensions['M'].width = 12.5
    sheet.column_dimensions['N'].width = 12.5
    sheet.column_dimensions['O'].width = 12.5
    sheet.column_dimensions['P'].width = 12.5
    sheet.column_dimensions['Q'].width = 12.5
    sheet.column_dimensions['R'].width = 12.5
    sheet.column_dimensions['S'].width = 12.5
    sheet.column_dimensions['T'].width = 12.5   
    sheet.column_dimensions['U'].width = 12.5   
    sheet.column_dimensions['V'].width = 12.5   
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(8, 21):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0.00_);[Red]("$"#,##0.00)'
    for col in range(1, 8):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=7).value == "Subtotal":
            for j in range(8,21):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 55
    sheet.freeze_panes = "H2"
    book.save(file)

In [135]:
def unmerge_rows_FFS(file):
    for col in range(1,5):
        rowstounmerge = []
        book = load_workbook(file)
        testsheet = book['ML & FFS Revenue'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                rowstounmerge.append(i)
        rowstounmerge.append(testsheet.max_row)
        for i in range(1,len(rowstounmerge)-1):        
            try:
                testsheet.unmerge_cells(start_row=rowstounmerge[i], start_column=col, end_row = rowstounmerge[i+1]-1, end_column=col) 
            except:
                pass
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['ML & FFS Revenue'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                continue
            else:
                testsheet.cell(row=i, column=col).value = testsheet.cell(row=i-1, column=col).value
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['ML & FFS Revenue'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=7).value == "Subtotal":
                testsheet.cell(row=i, column=col).value = None
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['ML & FFS Revenue'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=1).value == "Grand":
                testsheet.cell(row=i, column=2).value = None
                testsheet.cell(row=i, column=3).value = None
                testsheet.cell(row=i, column=4).value = None
        book.save(file) 

In [136]:
def format_FFS_pivot(Group, str):
    step1 = create_FFS_pivot(Group)
    step2 = add_subtotal_FFS(step1)
    to_excel_FFS(step2, path+str)
    edit_workbook_FFS(path+str)
    unmerge_rows_FFS(path+str)

## 8.  Add ML and FFS Pivots to Second Tab

In [137]:
'''
format_FFS_pivot(Developing_Economies_RollupFFS, '\\Developing_Economies_'+timestamp+".xlsx")
format_FFS_pivot(IndiaFFS, '\\India_'+timestamp+".xlsx")
format_FFS_pivot(OperationsFFS, '\\Operations_'+timestamp+".xlsx")
format_FFS_pivot(China_RollupFFS, '\\China_'+timestamp+".xlsx")
format_FFS_pivot(Breakthrough_Technology_FFS, '\\Breakthrough_Technology-'+timestamp+".xlsx")
'''
format_FFS_pivot(CarbonFree_MobilityFFS, '\\CarbonFree_Mobility_'+timestamp+".xlsx")
'''
format_FFS_pivot(ClimateAligned_IndustriesFFS, '\\Climate_Aligned_Industries_'+timestamp+".xlsx")
format_FFS_pivot(CarbonFreeBuildings_RollupFFS, '\\CarbonFree_Buildings_'+timestamp+".xlsx")
format_FFS_pivot(Strategic_Engagement_and_Analysis_GroupFFS, '\\Strategic_Engagement_and_Analysis_Group_'+timestamp+".xlsx")
format_FFS_pivot(UrbanTrans_FFS, '\\Urban_Transformation_'+timestamp+".xlsx")
format_FFS_pivot(USFFS, '\\US_'+timestamp+".xlsx")
format_FFS_pivot(Watttime_FFS, '\\WattTime_'+timestamp+".xlsx")
format_FFS_pivot(Carbon_Free_ElectricityFFS, '\\CarbonFree_Electricity_'+timestamp+".xlsx")
format_FFS_pivot(UrbanTrans_FFS, '\\Urban_Transformation_'+timestamp+".xlsx")
format_FFS_pivot(ClimateIntelFFS, '\\Climate_Intelligence_'+timestamp+".xlsx")
format_FFS_pivot(IndustryFFS, '\\Industry_'+timestamp+".xlsx")
format_FFS_pivot(GlobalClimate_FinanceFFS, '\\Global_Climate_Finance_'+timestamp+".xlsx")
format_FFS_pivot(Breakthrough_Technology_FFS, '\\Breakthrough_Technology-'+timestamp+".xlsx")
'''

'\nformat_FFS_pivot(ClimateAligned_IndustriesFFS, \'\\Climate_Aligned_Industries_\'+timestamp+".xlsx")\nformat_FFS_pivot(CarbonFreeBuildings_RollupFFS, \'\\CarbonFree_Buildings_\'+timestamp+".xlsx")\nformat_FFS_pivot(Strategic_Engagement_and_Analysis_GroupFFS, \'\\Strategic_Engagement_and_Analysis_Group_\'+timestamp+".xlsx")\nformat_FFS_pivot(UrbanTrans_FFS, \'\\Urban_Transformation_\'+timestamp+".xlsx")\nformat_FFS_pivot(USFFS, \'\\US_\'+timestamp+".xlsx")\nformat_FFS_pivot(Watttime_FFS, \'\\WattTime_\'+timestamp+".xlsx")\nformat_FFS_pivot(Carbon_Free_ElectricityFFS, \'\\CarbonFree_Electricity_\'+timestamp+".xlsx")\nformat_FFS_pivot(UrbanTrans_FFS, \'\\Urban_Transformation_\'+timestamp+".xlsx")\nformat_FFS_pivot(ClimateIntelFFS, \'\\Climate_Intelligence_\'+timestamp+".xlsx")\nformat_FFS_pivot(IndustryFFS, \'\\Industry_\'+timestamp+".xlsx")\nformat_FFS_pivot(GlobalClimate_FinanceFFS, \'\\Global_Climate_Finance_\'+timestamp+".xlsx")\nformat_FFS_pivot(Breakthrough_Technology_FFS, \'\\Bre

# 9.  Total Expenses

In [138]:
def create_df_allExpenses(program):
    firstdf = df1[df1['Program']==program]
    seconddf = firstdf[(firstdf['Account Digit']=='7')|(firstdf['Account Digit']=='8')]
    if seconddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return seconddf

In [139]:
DevelopmentExp = create_df_allExpenses("Development")
IslandsExp = create_df_allExpenses("Islands")
IndiaExp = create_df_allExpenses("India")
OperationsExp = create_df_allExpenses("Operations")
ChinaExp = create_df_allExpenses("China")
Breakthrough_Technology_Exp = create_df_allExpenses("Breakthrough Technology")
CarbonFree_MobilityExp = create_df_allExpenses("Carbon-Free Mobility")
ClimateAligned_IndustriesExp = create_df_allExpenses("Climate Aligned Industries")
CommunicationsExp = create_df_allExpenses("Communications")
GlobalClimate_FinanceExp = create_df_allExpenses("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupExp = create_df_allExpenses("Strategic Engagement and Analysis Group")
Carbon_Free_BuildingsExp = create_df_allExpenses("Carbon-Free Buildings")
Watttime_Exp = create_df_allExpenses("WattTime")
Carbon_Free_ElectricityExp = create_df_allExpenses("Carbon-Free Electricity")
UrbanTrans_Exp = create_df_allExpenses("Urban Transformation")
ClimateIntelExp = create_df_allExpenses("Climate Intelligence")
IndustryExp = create_df_allExpenses("Industry")
SEAsiaExp = create_df_allExpenses("SE Asia")
AfricaExp = create_df_allExpenses("Africa")
USExp = create_df_allExpenses("US")
BuildingElectrificationExp = create_df_allExpenses("Building Electrification")
Energy_Transition_AcademyExp = create_df_allExpenses("Energy Transition Academy")
BeijingExp = create_df_allExpenses("Beijing")

In [140]:
Developing_Economies_RollupExp = pd.concat([IslandsExp, AfricaExp, SEAsiaExp, Energy_Transition_AcademyExp], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupExp = pd.concat([Carbon_Free_BuildingsExp, BuildingElectrificationExp], ignore_index = True, sort=False)
China_RollupExp = pd.concat([ChinaExp, BeijingExp], ignore_index = True, sort=False)

In [141]:
def create_pivot_Exp(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant', 'GL Account', 'Memo', 'Transaction Number', 'Vendor Name', 'Vendor Rate', 'Vendor Contract Expiration Date'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [142]:
def add_subtotal_Exp(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', '', '', '', '', 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '', '', '', '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [143]:
def to_excel_Exp(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="All Expense Detail")
    writer.save()
    writer.close()    

In [144]:
def edit_workbook_Exp(file):
    book = load_workbook(file)
    sheet = book['All Expense Detail'] #active means last opened sheet
    sheet['A1']= "Initiative"
    sheet['B1']= "Project"
    sheet['C1']= "Grant"
    sheet['D1']= "GL Account"
    sheet['E1']= "Memo"
    sheet['F1']= "Transaction Number"
    sheet['G1']= "Vendor Name"
    sheet['H1']= "Vendor Rate"
    sheet['I1']= "Vendor Contract Expiration Date" 
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet['F1'].font = Font(bold=True)
    sheet['G1'].font = Font(bold=True)
    sheet['H1'].font = Font(bold=True)
    sheet['I1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 20
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 12.5
    sheet.column_dimensions['K'].width = 12.5
    sheet.column_dimensions['L'].width = 12.5
    sheet.column_dimensions['M'].width = 12.5
    sheet.column_dimensions['N'].width = 12.5
    sheet.column_dimensions['O'].width = 12.5
    sheet.column_dimensions['P'].width = 12.5
    sheet.column_dimensions['Q'].width = 12.5
    sheet.column_dimensions['R'].width = 12.5
    sheet.column_dimensions['S'].width = 12.5
    sheet.column_dimensions['T'].width = 12.5   
    sheet.column_dimensions['U'].width = 12.5   
    sheet.column_dimensions['V'].width = 12.5   
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(10, 23):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0.00_);[Red]("$"#,##0.00)'
    for col in range(1, 10):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=9).value == "Subtotal":
            for j in range(10,23):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 55
    sheet.freeze_panes = "J2"
    book.save(file)

In [145]:
def unmerge_rows_EXP(file):
    for col in range(1,5):
        rowstounmerge = []
        book = load_workbook(file)
        testsheet = book['All Expense Detail'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                rowstounmerge.append(i)
        rowstounmerge.append(testsheet.max_row)
        for i in range(1,len(rowstounmerge)-1):        
            try:
                testsheet.unmerge_cells(start_row=rowstounmerge[i], start_column=col, end_row = rowstounmerge[i+1]-1, end_column=col) 
            except:
                pass
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['All Expense Detail'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                continue
            else:
                testsheet.cell(row=i, column=col).value = testsheet.cell(row=i-1, column=col).value
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['All Expense Detail'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=9).value == "Subtotal":
                testsheet.cell(row=i, column=col).value = None
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['All Expense Detail'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=1).value == "Grand":
                testsheet.cell(row=i, column=2).value = None
                testsheet.cell(row=i, column=3).value = None
                testsheet.cell(row=i, column=4).value = None
        book.save(file) 

In [146]:
def format_Exp_pivot(Group, str):
    step1 = create_pivot_Exp(Group)
    step2 = add_subtotal_Exp(step1)
    to_excel_Exp(step2, path+str)
    edit_workbook_Exp(path+str)
    unmerge_rows_EXP(path+str)

In [147]:
'''
format_Exp_pivot(Developing_Economies_RollupExp, '\\Developing_Economies_'+timestamp+".xlsx")
format_Exp_pivot(IndiaExp, '\\India_'+timestamp+".xlsx")
format_Exp_pivot(OperationsExp, '\\Operations_'+timestamp+".xlsx")
format_Exp_pivot(China_RollupExp, '\\China_'+timestamp+".xlsx")
format_Exp_pivot(Breakthrough_Technology_Exp, '\\Breakthrough_Technology-'+timestamp+".xlsx")
'''
format_Exp_pivot(CarbonFree_MobilityExp, '\\CarbonFree_Mobility_'+timestamp+".xlsx")
'''
format_Exp_pivot(ClimateAligned_IndustriesExp, '\\Climate_Aligned_Industries_'+timestamp+".xlsx")
format_Exp_pivot(USExp, '\\US_'+timestamp+".xlsx")
format_Exp_pivot(UrbanTrans_Exp, '\\Urban_Transformation_'+timestamp+".xlsx")
format_Exp_pivot(Strategic_Engagement_and_Analysis_GroupExp, '\\Strategic_Engagement_and_Analysis_Group_'+timestamp+".xlsx")
format_Exp_pivot(CarbonFreeBuildings_RollupExp, '\\CarbonFree_Buildings_'+timestamp+".xlsx")
format_Exp_pivot(Watttime_Exp, '\\WattTime_'+timestamp+".xlsx")
format_Exp_pivot(Carbon_Free_ElectricityExp, '\\CarbonFree_Electricity_'+timestamp+".xlsx")
format_Exp_pivot(ClimateIntelExp, '\\Climate_Intelligence_'+timestamp+".xlsx")
format_Exp_pivot(IndustryExp, '\\Industry_'+timestamp+".xlsx")
format_Exp_pivot(GlobalClimate_FinanceExp, '\\Global_Climate_Finance_'+timestamp+".xlsx")
'''

'\nformat_Exp_pivot(ClimateAligned_IndustriesExp, \'\\Climate_Aligned_Industries_\'+timestamp+".xlsx")\nformat_Exp_pivot(USExp, \'\\US_\'+timestamp+".xlsx")\nformat_Exp_pivot(UrbanTrans_Exp, \'\\Urban_Transformation_\'+timestamp+".xlsx")\nformat_Exp_pivot(Strategic_Engagement_and_Analysis_GroupExp, \'\\Strategic_Engagement_and_Analysis_Group_\'+timestamp+".xlsx")\nformat_Exp_pivot(CarbonFreeBuildings_RollupExp, \'\\CarbonFree_Buildings_\'+timestamp+".xlsx")\nformat_Exp_pivot(Watttime_Exp, \'\\WattTime_\'+timestamp+".xlsx")\nformat_Exp_pivot(Carbon_Free_ElectricityExp, \'\\CarbonFree_Electricity_\'+timestamp+".xlsx")\nformat_Exp_pivot(ClimateIntelExp, \'\\Climate_Intelligence_\'+timestamp+".xlsx")\nformat_Exp_pivot(IndustryExp, \'\\Industry_\'+timestamp+".xlsx")\nformat_Exp_pivot(GlobalClimate_FinanceExp, \'\\Global_Climate_Finance_\'+timestamp+".xlsx")\n'

## 10. Add Mavenlink Detail

In [148]:
def create_Mavenlinkprogram(program):
    firstdf = dfMavenlink[dfMavenlink['Program']==program]
    if firstdf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return firstdf

In [149]:

DevelopmentML = create_Mavenlinkprogram("Development")
IslandsML = create_Mavenlinkprogram("Islands")
IndiaML = create_Mavenlinkprogram("India")
OperationsML = create_Mavenlinkprogram("Operations")
ChinaML = create_Mavenlinkprogram("China")
Breakthrough_Technology_ML = create_Mavenlinkprogram("Breakthrough Technology")
CarbonFree_MobilityML = create_Mavenlinkprogram("Carbon-Free Mobility")
ClimateAligned_IndustriesML = create_Mavenlinkprogram("Climate Aligned Industries")
CommunicationsML = create_Mavenlinkprogram("Communications")
GlobalClimate_FinanceML = create_Mavenlinkprogram("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupML = create_Mavenlinkprogram("Strategic Engagement and Analysis Group")
Carbon_Free_BuildingsML = create_Mavenlinkprogram("Carbon-Free Buildings")
Carbon_Free_ElectricityML = create_Mavenlinkprogram("Carbon-Free Electricity")
UrbanTrans_ML = create_Mavenlinkprogram("Urban Transformation")
ClimateIntelML = create_Mavenlinkprogram("Climate Intelligence")
IndustryML = create_Mavenlinkprogram("Industry")
SEAsiaML = create_Mavenlinkprogram("SE Asia")
AfricaML = create_Mavenlinkprogram("Africa")
USML = create_Mavenlinkprogram("US")
BuildingElectrificationML = create_Mavenlinkprogram("Building Electrification")
Energy_Transition_AcademyML = create_Mavenlinkprogram("Energy Transition Academy")
BeijingML = create_Mavenlinkprogram("Beijing")



Beijing DF is EMPTY


In [150]:
Developing_Economies_RollupML = pd.concat([IslandsML, AfricaML, SEAsiaML, Energy_Transition_AcademyML], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupML = pd.concat([Carbon_Free_BuildingsML, BuildingElectrificationML], ignore_index = True, sort=False)
China_RollupML = pd.concat([ChinaML, BeijingML], ignore_index = True, sort=False)

In [151]:
def create_ML_pivot(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Employee','Case/Task/Event','Grant','Billable'], values = 'Percent of Total Monthly Hours', columns = 'Month', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [152]:
def to_excel_ML(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Mavenlink Hours")
    writer.save()
    writer.close()   

In [153]:
def edit_workbook_ML(file):
    book = load_workbook(file)
    sheet = book["Mavenlink Hours"] #active means last opened sheet
    sheet['B1']= "Employee"
    sheet['A1']= "Initiative"
    sheet['C1']= "Case/Task/Event"    
    sheet['D1']= "Grant"     
    sheet['E1']= "Billable"    
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 25
    sheet.column_dimensions['D'].width = 25
    sheet.column_dimensions['E'].width = 25
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 15
    sheet.column_dimensions['Q'].width = 15
    sheet.column_dimensions['R'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(6, 19):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '0.00%'
    for col in range(1, 6):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill 
    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 60
    sheet.freeze_panes = "F2"
    book.save(file)

In [154]:
def unmerge_rows_ML(file):
    for col in range(1,4):
        rowstounmerge = []
        book = load_workbook(file)
        testsheet = book["Mavenlink Hours"] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                rowstounmerge.append(i)
        rowstounmerge.append(testsheet.max_row+1)
        for i in range(1,len(rowstounmerge)-1):        
            try:
                testsheet.unmerge_cells(start_row=rowstounmerge[i], start_column=col, end_row = rowstounmerge[i+1]-1, end_column=col) 
            except:
                pass
        book.save(file) 
        book = load_workbook(file)
        testsheet = book["Mavenlink Hours"] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                continue
            else:
                testsheet.cell(row=i, column=col).value = testsheet.cell(row=i-1, column=col).value
        book.save(file) 


In [155]:
def format_pivot_ML(Group, str):
    step1 = create_ML_pivot(Group)
    to_excel_ML(step1, path+str)
    edit_workbook_ML(path+str)
    unmerge_rows_ML(path+str)

In [156]:
'''
format_pivot_ML(Developing_Economies_RollupML, '\\Developing_Economies_'+timestamp+".xlsx")
format_pivot_ML(IndiaML, '\\India_'+timestamp+".xlsx")
format_pivot_ML(OperationsML, '\\Operations_'+timestamp+".xlsx")
format_pivot_ML(China_RollupML, '\\China_'+timestamp+".xlsx")
format_pivot_ML(Breakthrough_Technology_ML, '\\Breakthrough_Technology-'+timestamp+".xlsx")
'''
format_pivot_ML(CarbonFree_MobilityML, '\\CarbonFree_Mobility_'+timestamp+".xlsx")
'''
format_pivot_ML(ClimateAligned_IndustriesML, '\\Climate Intelligence_'+timestamp+".xlsx")
format_pivot_ML(Strategic_Engagement_and_Analysis_GroupML, '\\Strategic_Engagement_and_Analysis_Group_'+timestamp+".xlsx")
format_pivot_ML(USML, '\\US_'+timestamp+".xlsx")
format_pivot_ML(UrbanTrans_ML, '\\Urban_Transformation_'+timestamp+".xlsx")
format_pivot_ML(CarbonFreeBuildings_RollupML, '\\CarbonFree_Buildings_'+timestamp+".xlsx")
format_pivot_ML(Carbon_Free_ElectricityML, '\\CarbonFree_Electricity_'+timestamp+".xlsx")
format_pivot_ML(ClimateIntelML, '\\Climate_Intelligence_'+timestamp+".xlsx")
format_pivot_ML(IndustryML, '\\Industry_'+timestamp+".xlsx")
format_pivot_ML(GlobalClimate_FinanceML, '\\Global_Climate_Finance_'+timestamp+".xlsx")
'''

'\nformat_pivot_ML(ClimateAligned_IndustriesML, \'\\Climate Intelligence_\'+timestamp+".xlsx")\nformat_pivot_ML(Strategic_Engagement_and_Analysis_GroupML, \'\\Strategic_Engagement_and_Analysis_Group_\'+timestamp+".xlsx")\nformat_pivot_ML(USML, \'\\US_\'+timestamp+".xlsx")\nformat_pivot_ML(UrbanTrans_ML, \'\\Urban_Transformation_\'+timestamp+".xlsx")\nformat_pivot_ML(CarbonFreeBuildings_RollupML, \'\\CarbonFree_Buildings_\'+timestamp+".xlsx")\nformat_pivot_ML(Carbon_Free_ElectricityML, \'\\CarbonFree_Electricity_\'+timestamp+".xlsx")\nformat_pivot_ML(ClimateIntelML, \'\\Climate_Intelligence_\'+timestamp+".xlsx")\nformat_pivot_ML(IndustryML, \'\\Industry_\'+timestamp+".xlsx")\nformat_pivot_ML(GlobalClimate_FinanceML, \'\\Global_Climate_Finance_\'+timestamp+".xlsx")\n'

# Create Reports for Heather and Jon Creyts

In [157]:
todayMD = datetime.today()
datemMD = datetime(today.year, today.month-1, 1)
timestampMD = datem.strftime("%B %Y")

In [158]:
timestampMD

'February 2021'

In [159]:
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment

CFOpath = r'C:\Users\Jeremy Wendt\anaconda3\envs\Code\Detail_Trans_Project_FY21\reports\CFO_Reports'

CFOTemplatepath = 'C:\\Users\\Jeremy Wendt\\anaconda3\\envs\\Code\\Detail_Trans_Project_FY21\\Data\\raw\\CFO_Cover_Page.xlsx'

In [160]:
blueFill = PatternFill(start_color='0033CCCC',
                   end_color='0033CCCC',
                   fill_type='solid')



## Create Cover Sheet CFO

In [161]:
Logopath = r'C:\Users\Jeremy Wendt\anaconda3\envs\Code\Detail_Trans_Project_FY21\notebooks\img\Picture1.png'

# 'C:\\Users\\Jeremy Wendt\\anaconda3\\envs\\Code\\Detail_Trans_Project_FY21\\notebooks\\img\\MD_Cover_Page.xlsx'

In [162]:
wbCFO = load_workbook(CFOTemplatepath)
wsCFO = wbCFO['Cover']    
img = openpyxl.drawing.image.Image(Logopath)
img.anchor = 'A1'
wsCFO.add_image(img)
wsCFO.cell(row = 14, column = 1).value = 'CFO Report ' + timestampMD
wbCFO.save(str(CFOpath+'\\CFO_Report_'+timestampMD+'.xlsx'))

## Restricted Revenue Spend Type by Grant

In [163]:
def create_CFO_pivot(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Grant','Inst or Reclass'], values = 'Restricted Revenue', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [164]:
def add_subtotal_CFO(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, 'Subtotal'))) for k, d in PivotedGroup.groupby('Grant')]).append((PivotedGroup.sum()).rename(('Grand', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [165]:
def to_excel_CFO(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="RR by Expense Type", startrow = 1)
    writer.save()
    writer.close()   

In [166]:
def edit_workbook_CFO(file):
    book = load_workbook(file)
    sheet = book["RR by Expense Type"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Grant"
    sheet['B2']= "Inst or Reclass" 
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:O1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['O1']
    top_left_cell.value = "Restricted Revenue by Expense Type"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="DDDDDD")
    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(b=True, color="FFFFFF")
    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(3, 16):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 3):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=2).value == "Subtotal":
            for j in range(2,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grant":
            for j in range(1,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(16, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "C3"
    book.save(file)

In [167]:
def format_multi_pivot_CFO(Group, str):
    step1 = create_CFO_pivot(Group)
    step2 = add_subtotal_CFO(step1)
    to_excel_CFO(step2, str)
    edit_workbook_CFO(str)

In [168]:
dfRestrictedONLY = df1[df1['Rev or Exp']=="Expense"]

In [169]:
format_multi_pivot_CFO(dfRestrictedONLY, CFOpath+'\\CFO_Report_'+timestampMD+'.xlsx')

## Unrestricted Use by Project Allocation and Program

In [170]:
def create_CFO_pivot2(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Project Allocations','Program'], values = 'Unrestricted', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [171]:
def add_subtotal_CFO2(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, 'Subtotal'))) for k, d in PivotedGroup.groupby('Project Allocations')]).append((PivotedGroup.sum()).rename(('Grand', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [172]:
def to_excel_CFO2(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Unrestricted by Project All", startrow = 1)
    writer.save()
    writer.close()    

In [173]:
def edit_workbook_CFO2(file):
    book = load_workbook(file)
    sheet = book["Unrestricted by Project All"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Project Allocations"
    sheet['B2']= 'Program'
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:O1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['O1']
    top_left_cell.value = "Unrestricted Used by Project Type"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="DDDDDD")
    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(b=True, color="FFFFFF")
    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(3, 16):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 3):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=2).value == "Subtotal":
            for j in range(2,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Project Allocations":
            for j in range(1,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(16, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "C3"
    book.save(file)

In [174]:
def format_multi_pivot_CFO2(Group, str):
    step1 = create_CFO_pivot2(Group)
    step2 = add_subtotal_CFO2(step1)
    to_excel_CFO2(step2, str)
    edit_workbook_CFO2(str)

In [175]:
format_multi_pivot_CFO2(df1, CFOpath+'\\CFO_Report_'+timestampMD+'.xlsx')

# Managing Director Dashboard

In [176]:
MDpath = r'C:\Users\Jeremy Wendt\anaconda3\envs\Code\Detail_Trans_Project_FY21\reports\MD_Package'


Templatepath = 'C:\\Users\\Jeremy Wendt\\anaconda3\\envs\\Code\\Detail_Trans_Project_FY21\\Data\\raw\\MD_Cover_Page.xlsx'

In [177]:

#wb = openpyxl.load_workbook(filename=Templatepath)
#ws = wb['Cover']

In [178]:
ProgramsMD = Programs

In [181]:
ProgramsMD.remove('na')
ProgramsMD.remove('Communications')
ProgramsMD.remove('Development')
ProgramsMD.remove('Building Electrification')
ProgramsMD.remove('Islands')
ProgramsMD.remove('SE Asia')
ProgramsMD.remove('Africa')
ProgramsMD.remove('Energy Transition Academy')
ProgramsMD.remove('Sustainable Finance')
ProgramsMD.remove('Energy Web Foundation')
ProgramsMD.remove('Business Renewable Center')
ProgramsMD.remove('Emerging Energy Solutions')
ProgramsMD.remove('Shine')
ProgramsMD.append('Developing Economies')
ProgramsMD.remove('Beijing')

In [182]:
ProgramsMD

['Carbon-Free Electricity',
 'Urban Transformation',
 'Operations',
 'China',
 'Industry',
 'Carbon-Free Buildings',
 'Strategic Engagement and Analysis Group',
 'Global Climate Finance',
 'India',
 'Carbon-Free Mobility',
 'WattTime',
 'Breakthrough Technology',
 'Climate Aligned Industries',
 'Climate Intelligence',
 'US',
 'Developing Economies']

## Create Cover Sheet

In [183]:
CoverNames = []

for i in ProgramsMD:
    CoverNames.append(i + " - " + timestampMD)

In [184]:
for i in CoverNames:    
    wb = load_workbook(Templatepath)
    ws = wb['Cover']    
    img = openpyxl.drawing.image.Image(Logopath)
    img.anchor = 'A1'
    ws.add_image(img)
    ws.cell(row = 14, column = 1).value = i
    wb.save(str(MDpath+'\\'+i+' - MD Package.xlsx'))

## Create First MD Table

In [185]:
def create_MD_pivot1(Group):
    Month_order = ['Jul 2018', 'Aug 2018', 'Sep 2018', 'Oct 2018', 'Nov 2018', 'Dec 2018', 'Jan 2019', 'Feb 2019', 'Mar 2019', 'Apr 2019', 'May 2019', 'Jun 2019', 'July 2019', 'Aug 2019', 'Sep 2019', 'Oct 2019', 'Nov 2019', 'Dec 2019', 'Jan 2020', 'Feb 2020', 'Mar 2020', 'Apr 2020', 'May 2020', 'Jun 2020', 'Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['FY21 Total'] = table2.iloc[:,-12:].sum(axis=1)
    table2['FY20-FY21 Total'] = table2.iloc[:,-36:].sum(axis=1)
    table2 = table2[table2['FY20-FY21 Total']!=0]      
    return table2

In [186]:
def add_subtotal_MD1(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k,'', 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand','', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [187]:
def to_excel_MD1(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Restricted Revenue Exp", startrow = 1)
    writer.save()
    writer.close()    

In [188]:
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment

def edit_workbook_MD1(file):
    book = load_workbook(file)
    sheet = book["Restricted Revenue Exp"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Project"
    sheet['C2']= "Grant"    
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:AN1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['AN1']
    top_left_cell.value = "FY21 Restricted Revenue"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="DDDDDD")
    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(b=True, color="FFFFFF")
    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 25
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 15
    sheet.column_dimensions['AO'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(4, 41):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 4):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=3).value == "Subtotal":
            for j in range(3,41):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,41):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,41):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(41, 42):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "D3"
    sheet.column_dimensions.group("D","AA",hidden=True)
    book.save(file)

In [189]:
def format_multi_pivot_MD1(Group, str):
    step1 = create_MD_pivot1(Group)
    step2 = add_subtotal_MD1(step1)
    to_excel_MD1(step2, MDpath+str)
    edit_workbook_MD1(MDpath+str)

In [190]:
CoverNames

['Carbon-Free Electricity - February 2021',
 'Urban Transformation - February 2021',
 'Operations - February 2021',
 'China - February 2021',
 'Industry - February 2021',
 'Carbon-Free Buildings - February 2021',
 'Strategic Engagement and Analysis Group - February 2021',
 'Global Climate Finance - February 2021',
 'India - February 2021',
 'Carbon-Free Mobility - February 2021',
 'WattTime - February 2021',
 'Breakthrough Technology - February 2021',
 'Climate Aligned Industries - February 2021',
 'Climate Intelligence - February 2021',
 'US - February 2021',
 'Developing Economies - February 2021']

In [191]:
format_multi_pivot_MD1(Developing_Economies_Rollup, '\\Developing Economies - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Climate_Aligned_Industries, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(India, '\\India - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Operations, '\\Operations - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(China_Rollup, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Breakthrough_Technology, '\\Breakthrough Technology - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(CarbonFree_Mobility, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(CarbonFree_Electricty, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Strategic_Engagement_and_Analysis_Group, '\\Strategic Engagement and Analysis Group - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(CarbonFreeBuildings_Rollup, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Urban_Transformation, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Climate_Intelligence, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Industry, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(US, '\\US - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(WattTime, '\\WattTime - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Global_Climate_Finance, '\\Global Climate Finance - '+timestampMD+' - MD Package.xlsx')

# Create Multi Lat Tab

In [192]:
dfFFSDataZeros = dfFFSDataZeros[['Program','Initiative','Project','Project Type', 'Contract Start Date', 'Contract End Date','Invoice Type', 'Invoicing Notes', 'Current Month Action', 'Revenue Recognition Notes', 'Contract Amount', 'Total Revenue Recognized', 'Reimbursables NTE', 'Reimbursables Actual', 'Remaining Revenue']]


ChinaFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="China"]
CFEFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Carbon-Free Electricity"]
UTFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Urban Transformation"]
CAIFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Climate Aligned Industries"]
CFBFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Carbon-Free Buildings"]


In [193]:
from openpyxl.utils.dataframe import dataframe_to_rows

def create_data_FFSZeroBalance(Group, str):
    table = Group[['Initiative','Project','Project Type', 'Contract Start Date', 'Contract End Date','Invoice Type', 'Invoicing Notes', 'Current Month Action', 'Revenue Recognition Notes', 'Contract Amount', 'Total Revenue Recognized', 'Reimbursables NTE', 'Reimbursables Actual', 'Remaining Revenue']]
    book = load_workbook(MDpath+str)
    sheet = book["ER & ML Govt Rev"]
    data = table
    rows = dataframe_to_rows(table, index=False, header=True)
    for r_idx, row in enumerate(rows, sheet.max_row+2):
        for c_idx, value in enumerate(row, 1):
            sheet.cell(row=r_idx, column=c_idx, value=value)
            sheet.cell(column=c_idx, row=r_idx).font = Font(bold=True)
            sheet.cell(column=c_idx, row=r_idx).number_format = '"$"#,##0;[Red]("$"#,##0)'
    book.save(MDpath+str)

In [194]:
def create_MD_pivot2(Group):
    Month_order = ['Jul 2018', 'Aug 2018', 'Sep 2018', 'Oct 2018', 'Nov 2018', 'Dec 2018', 'Jan 2019', 'Feb 2019', 'Mar 2019', 'Apr 2019', 'May 2019', 'Jun 2019', 'July 2019', 'Aug 2019', 'Sep 2019', 'Oct 2019', 'Nov 2019', 'Dec 2019', 'Jan 2020', 'Feb 2020', 'Mar 2020', 'Apr 2020', 'May 2020', 'Jun 2020', 'Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Project Type', 'Contract Start Date', 'Contract End Date','Invoice Type', 'Invoicing Notes', 'Current Month Action', 'Revenue Recognition Notes', 'Contract Amount', 'Total Revenue Recognized', 'Reimbursables NTE', 'Reimbursables Actual', 'Remaining Revenue'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['FY21 Total'] = table2.iloc[:,-12:].sum(axis=1)
    table2['FY20-FY21 Total'] = table2.iloc[:,-36:].sum(axis=1)
    table2 = table2[table2['FY20-FY21 Total']!=0]
    return table2

In [195]:
def add_subtotal_MD2(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '','','','','','','','','','','','','Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand', '','','','','','','','','','','','', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [196]:
def to_excel_MD2(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book
   
    df.to_excel(writer, sheet_name="ER & ML Govt Rev", startrow = 1)
    writer.save()
    writer.close()    

In [197]:
def edit_workbook_MD2(file):
    book = load_workbook(file)
    sheet = book["ER & ML Govt Rev"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Project"
    sheet['C2']= "Project Type"
    sheet['D2']= "Contract Start Date"
    sheet['E2']= "Contract End Date"
    sheet['F2']= "Invoice Type"
    sheet['G2']= "Invoicing Notes"
    sheet['H2']= "Current Month Action"
    sheet['I2']= "Revenue Recognition Notes"
    sheet['J2']= "Contract Amount"
    sheet['K2']= "Total Revenue Recognized (INCLUDES DEFERRED REVENUE)"
    sheet['L2']= "Reimbursables NTE"
    sheet['M2']= "Reimbursables Actual"
    sheet['N2']= "Remaining Revenue"
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:AY1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['AY1']
    top_left_cell.value = "FY21 Earned, Government, and Multilateral Grant Revenue"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="DDDDDD")
    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(b=True, color="FFFFFF")
    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 25
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['D'].hidden = True
    sheet.column_dimensions['L'].hidden = True
    sheet.column_dimensions['M'].hidden = True
    sheet.column_dimensions['F'].hidden = True
    sheet.column_dimensions['H'].hidden = True
    sheet.column_dimensions['AZ'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(1, 15):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for col in range(10, 52):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=14).value == "Subtotal":
            for j in range(2,52):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,52):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,52):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(52, 53):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "O3"
    sheet.column_dimensions.group("O","AL",hidden=True)
    book.save(file)

In [198]:
def format_multi_pivot_MD2(Group, str):
    step1 = create_MD_pivot2(Group)
    step2 = add_subtotal_MD2(step1)
    to_excel_MD2(step2, MDpath+str)
    edit_workbook_MD2(MDpath+str)

In [199]:
format_multi_pivot_MD2(Developing_Economies_RollupFFS, '\\Developing Economies - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(ClimateAligned_IndustriesFFS, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(IndiaFFS, '\\India - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(OperationsFFS, '\\Operations - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(ChinaFFS, '\\China - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD2(Breakthrough_Technology_FFS, '\\Breakthrough Technology - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(CarbonFree_MobilityFFS, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(Carbon_Free_ElectricityFFS, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD2(Strategic_Engagement_and_Analysis_GroupFFS, '\\Strategic Engagement and Analysis Group - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(CarbonFreeBuildings_RollupFFS, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(UrbanTrans_FFS, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(ClimateIntelFFS, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(IndustryFFS, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(GlobalClimate_FinanceFFS, '\\Global Climate Finance - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(Watttime_FFS, '\\WattTime - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD2(USFFS, '\\US - '+timestampMD+".xlsx")

In [200]:
create_data_FFSZeroBalance(CFEFFSNOREV,'\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance(ChinaFFSNOREV,'\\China - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance(UTFFSNOREV,'\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance(CAIFFSNOREV,'\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance(CFBFFSNOREV,'\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')

# Create All Expenses Report

In [201]:
def create_MD_pivot3(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Rollup Account'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [202]:
def add_subtotal_MD3(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [203]:
def to_excel_MD3(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Total Expenses", startrow = 1)
    writer.save()
    writer.close()    

In [204]:
def edit_workbook_MD3(file):
    book = load_workbook(file)
    sheet = book["Total Expenses"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Account Category"
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:O1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['O1']
    top_left_cell.value = "FY21 Total Program Expenses by Initiative and Account"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="DDDDDD")
    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(b=True, color="FFFFFF")
    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(3, 16):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 3):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=2).value == "Subtotal":
            for j in range(2,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(16, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "C3"
    book.save(file)

In [205]:
def format_multi_pivot_MD3(Group, str):
    step1 = create_MD_pivot3(Group)
    step2 = add_subtotal_MD3(step1)
    to_excel_MD3(step2, MDpath+str)
    edit_workbook_MD3(MDpath+str)

In [206]:
format_multi_pivot_MD3(Developing_Economies_RollupExp, '\\Developing Economies - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(ClimateAligned_IndustriesExp, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(IndiaExp, '\\India - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(OperationsExp, '\\Operations - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(ChinaExp, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(Breakthrough_Technology_Exp, '\\Breakthrough Technology - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(CarbonFree_MobilityExp, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(Carbon_Free_ElectricityExp, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(Strategic_Engagement_and_Analysis_GroupExp, '\\Strategic Engagement and Analysis Group - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(CarbonFreeBuildings_RollupExp, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(UrbanTrans_Exp, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(ClimateIntelExp, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(IndustryExp, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(USExp, '\\US - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(GlobalClimate_FinanceExp, '\\Global Climate Finance - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(Watttime_Exp, '\\WattTime - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD3(BuildingElectrificationExp, '\\Building Electrification - '+timestampMD+' - MD Package.xlsx')

# Unrestricted Report

In [207]:
def create_Totalprogram(program):
    firstdf = df1[df1['Program']==program]
    if firstdf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return firstdf

In [208]:
DevelopmentTotal = create_Totalprogram('Development')
IslandsTotal = create_Totalprogram('Islands')
IndiaTotal = create_Totalprogram('India')
OperationsTotal = create_Totalprogram('Operations')
ChinaTotal = create_Totalprogram('China')
Breakthrough_TechnologyTotal = create_Totalprogram('Breakthrough Technology')
CarbonFree_MobilityTotal = create_Totalprogram('Carbon-Free Mobility')
Climate_Aligned_IndustriesTotal = create_Totalprogram('Climate Aligned Industries')
Global_Climate_FinanceTotal = create_Totalprogram('Global Climate Finance')
Strategic_Engagement_and_Analysis_GroupTotal = create_Totalprogram('Strategic Engagement and Analysis Group')
CarbonFree_BuildingsTotal = create_Totalprogram('Carbon-Free Buildings')
WattTimeTotal = create_Totalprogram('WattTime')
CarbonFree_ElectrictyTotal = create_Totalprogram('Carbon-Free Electricity')
Urban_TransformationTotal = create_Totalprogram('Urban Transformation')
Climate_IntelligenceTotal = create_Totalprogram('Climate Intelligence')
IndustryTotal = create_Totalprogram('Industry')
SE_AsiaTotal = create_Totalprogram('SE Asia')
AfricaTotal = create_Totalprogram('Africa')
USTotal = create_Totalprogram('US')
Building_ElectrificationTotal = create_Totalprogram('Building Electrification')
Energy_Transition_AcademyTotal = create_Totalprogram('Energy Transition Academy')
BeijingTotal = create_Totalprogram('Beijing')

In [209]:
Developing_Economies_RollupTotal = pd.concat([IslandsTotal, AfricaTotal, SE_AsiaTotal, Energy_Transition_AcademyTotal], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupTotal = pd.concat([CarbonFree_BuildingsTotal, Building_ElectrificationTotal], ignore_index = True, sort=False)
China_RollupTotal = pd.concat([ChinaTotal, BeijingTotal], ignore_index = True, sort=False)

In [210]:
def create_MD_pivot4(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant'], values = 'Unrestricted', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [211]:
def add_subtotal_MD4(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k,'', 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand','', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [212]:
def to_excel_MD4(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Unrestricted Use", startrow = 1)
    writer.save()
    writer.close()    

In [213]:
def edit_workbook_MD4(file):
    book = load_workbook(file)
    sheet = book["Unrestricted Use"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Project"
    sheet['C2']= "Grant"    
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:P1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['P1']
    top_left_cell.value = "FY21 Unrestricted Use by Initiative, Project, and Grant"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="DDDDDD")
    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(b=True, color="FFFFFF")
    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 25
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(4, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 4):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=3).value == "Subtotal":
            for j in range(3,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin) 
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(17, 18):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "D3"
    book.save(file)

In [214]:
def format_multi_pivot_MD4(Group, str):
    step1 = create_MD_pivot4(Group)
    step2 = add_subtotal_MD4(step1)
    to_excel_MD4(step2, MDpath+str)
    edit_workbook_MD4(MDpath+str)

In [215]:
format_multi_pivot_MD4(Developing_Economies_RollupTotal, '\\Developing Economies - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Climate_Aligned_IndustriesTotal, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(IndiaTotal, '\\India - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(OperationsTotal, '\\Operations - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(China_RollupTotal, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Breakthrough_TechnologyTotal, '\\Breakthrough Technology - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(CarbonFree_MobilityTotal, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(CarbonFree_ElectrictyTotal, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Strategic_Engagement_and_Analysis_GroupTotal, '\\Strategic Engagement and Analysis Group - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(CarbonFreeBuildings_RollupTotal, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Urban_TransformationTotal, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Climate_IntelligenceTotal, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(IndustryTotal, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(USTotal, '\\US - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(WattTimeTotal, '\\WattTime - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Global_Climate_FinanceTotal, '\\Global Climate Finance - '+timestampMD+' - MD Package.xlsx')


## Create Salesforce Pivots for Philanthropic

In [216]:
def create_Salesforceprogram(program):
    firstdf = dfSalesforce[dfSalesforce['Program: Program Name']==program]
    seconddf = firstdf[firstdf['Grant Status']!='Complete - Satisfactory']
    if seconddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return firstdf

In [217]:
DevelopmentSF = create_Salesforceprogram("General Support")
IslandsSF = create_Salesforceprogram("Islands")
IndiaSF = create_Salesforceprogram("India")
OperationsSF = create_Salesforceprogram("Operations")
ChinaSF = create_Salesforceprogram("China")
Breakthrough_Technology_SF = create_Salesforceprogram("Breakthrough Technology")
CarbonFree_MobilitySF = create_Salesforceprogram("Carbon-Free Mobility")
ClimateAligned_IndustriesSF = create_Salesforceprogram("Climate Aligned Industries")
#CommunicationsSF = create_Salesforceprogram("Communications")
GlobalClimate_FinanceSF = create_Salesforceprogram("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupSF = create_Salesforceprogram("Strategic Engagement and Analysis Group")
Carbon_Free_BuildingsSF = create_Salesforceprogram("Carbon-Free Buildings")
Carbon_Free_ElectricitySF = create_Salesforceprogram("Carbon-Free Electricity")
UrbanTrans_SF = create_Salesforceprogram("Urban Transformation")
ClimateIntelSF = create_Salesforceprogram("Climate Intelligence")
IndustrySF = create_Salesforceprogram("Industry")
SEAsiaSF = create_Salesforceprogram("SE Asia Energy")
AfricaSF = create_Salesforceprogram("Africa")
USSF = create_Salesforceprogram("US")
BuildingElectrificationSF = create_Salesforceprogram("Building Electrification")
Energy_Transition_AcademySF = create_Salesforceprogram("Energy Transition Academy")
BeijingSF = create_Salesforceprogram("Beijing")

Building Electrification DF is EMPTY
Beijing DF is EMPTY


In [218]:
Developing_Economies_RollupSF = pd.concat([IslandsSF, AfricaSF, SEAsiaSF, Energy_Transition_AcademySF], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupSF = pd.concat([Carbon_Free_BuildingsSF, BuildingElectrificationSF], ignore_index = True, sort=False)
China_RollupSF = pd.concat([ChinaSF, BeijingSF], ignore_index = True, sort=False)

In [219]:
def create_MD_pivot5(Group):
    table = pd.pivot_table(Group, index = ['Opportunity Record Type','Initiative','Forecast Category','Opportunity Name', 'RMI Grant Name', 'Revenue Name','Stage', 'Revenue Date', 'Grant Term - Start Date', 'Grant Term - End Date', 'Revenue Status', 'Program Fund Type', 'Probability (%)', 'Revenue Amount'], values = 'Expected Revenue', aggfunc=np.sum)
    return table

In [220]:
today = datetime.today()
timestamptoday = today.strftime("%B %d, %Y")

In [221]:
def to_excel_MD5(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="SF Data DEV&GovMLBL", startrow = 1)
    writer.save()
    writer.close()   

In [222]:
def edit_workbook_MD5(file):
    book = load_workbook(file)
    sheet = book["SF Data DEV&GovMLBL"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Opportunity Record Type"    
    sheet['B2']= "Initiative"
    sheet['C2']= "Forecast Category" 
    sheet['D2']= "Opportunity Name"
    sheet['E2']= "RMI Grant Name"    
    sheet['F2']= "Revenue Name" 
    sheet['G2']= "Stage"     
    sheet['H2']= "Revenue Date"     
    sheet['I2']= "Grant Term - Start Date"
    sheet['J2']= "Grant Term - End Date"  
    sheet['K2']= "Revenue Status"   
    sheet['L2']= "Program Fund Type"   
    sheet['M2']= "Probability (%)"     
    sheet['N2']= "Revenue Amount"
    sheet['O2']= "Expected Revenue"   
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    sheet['D2'].font = Font(bold=True) 
    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:O1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['O1']
    top_left_cell.value = "SalesForce FY20+ Data as of " +timestamptoday
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="DDDDDD")
    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(b=True, color="FFFFFF")
    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 25
    sheet.column_dimensions['F'].width = 25
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['E'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(13, 16):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '#,##0;[Red](#,##0)'
    for col in range(1, 7):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Opportunity Record Type":
            for j in range(1,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(15, 16):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(top=thin, left=thin, right=thin, bottom=thin)
    for col in range(7, 16):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).font = Font(bold=False)
    sheet.sheet_view.zoomScale = 85  
    sheet.freeze_panes = sheet['G3']
    book.save(file)

In [223]:
def format_multi_pivot_MD5(Group, str):
    step1 = create_MD_pivot5(Group)
    to_excel_MD5(step1, MDpath+str)
    edit_workbook_MD5(MDpath+str)

In [224]:
format_multi_pivot_MD5(Developing_Economies_RollupSF, '\\Developing Economies - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(ClimateAligned_IndustriesSF, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(IndiaSF, '\\India - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(OperationsSF, '\\Operations - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(China_RollupSF, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(Breakthrough_Technology_SF, '\\Breakthrough Technology - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(CarbonFree_MobilitySF, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(Carbon_Free_ElectricitySF, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(Strategic_Engagement_and_Analysis_GroupSF, '\\Strategic Engagement and Analysis Group - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(CarbonFreeBuildings_RollupSF, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(UrbanTrans_SF, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(ClimateIntelSF, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(IndustrySF, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(USSF, '\\US - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD6(BuildingElectrificationSF, '\\Building Electrification - '+timestampMD+' - MD Package.xlsx')

## Create Salesforce Pivots for FFS

In [225]:
def create_Salesforceprogram2(program):
    firstdf = dfSalesforceFFS[dfSalesforceFFS['Program Area: Program Name']==program]
    if firstdf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return firstdf

In [226]:
DevelopmentSFffs = create_Salesforceprogram2("General Support")
IslandsSFffs = create_Salesforceprogram2("Islands")
IndiaSFffs = create_Salesforceprogram2("India")
OperationsSFffs = create_Salesforceprogram2("Operations")
ChinaSFffs = create_Salesforceprogram2("China")
Breakthrough_Technology_SFffs = create_Salesforceprogram2("Breakthrough Technology")
CarbonFree_MobilitySFffs = create_Salesforceprogram2("Carbon-Free Mobility")
ClimateAligned_IndustriesSFffs = create_Salesforceprogram2("Climate Aligned Industries")
#CommunicationsSF = create_Salesforceprogram("Communications")
GlobalClimate_FinanceSFffs = create_Salesforceprogram2("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupSFffs = create_Salesforceprogram2("Strategic Engagement and Analysis Group")
Carbon_Free_BuildingsSFffs = create_Salesforceprogram2("Carbon-Free Buildings")
Carbon_Free_ElectricitySFffs = create_Salesforceprogram2("Carbon-Free Electricity")
UrbanTrans_SFffs = create_Salesforceprogram2("Urban Transformation")
ClimateIntelSFffs = create_Salesforceprogram2("Climate Intelligence")
IndustrySFffs = create_Salesforceprogram2("Industry")
SEAsiaSFffs = create_Salesforceprogram2("SE Asia Energy")
AfricaSFffs = create_Salesforceprogram2("Africa")
USSFffs = create_Salesforceprogram2("US")
BuildingElectrificationSFffs = create_Salesforceprogram2("Building Electrification")
Energy_Transition_AcademySFffs = create_Salesforceprogram2("Energy Transition Academy")
BeijingSFffs = create_Salesforceprogram2("Beijing")

General Support DF is EMPTY
Operations DF is EMPTY
Global Climate Finance DF is EMPTY
SE Asia Energy DF is EMPTY
Building Electrification DF is EMPTY
Energy Transition Academy DF is EMPTY
Beijing DF is EMPTY


In [227]:
Developing_Economies_RollupSFffs = pd.concat([IslandsSFffs, AfricaSFffs, SEAsiaSFffs, Energy_Transition_AcademySFffs], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupSFffs = pd.concat([Carbon_Free_BuildingsSFffs, BuildingElectrificationSFffs], ignore_index = True, sort=False)
China_RollupSFffs = pd.concat([ChinaSFffs, BeijingSFffs], ignore_index = True, sort=False)

In [228]:
def create_MD_pivot6(Group):
    table = pd.pivot_table(Group, index = ['Initiative','Account Name', 'Opportunity Name','Relationship Manager', 'Stage', 'Close Date', 'Probability (%)', 'Amount'], values = 'Weighted Value of Opportunity', aggfunc=np.sum)
    return table

In [229]:
def to_excel_MD6(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="SF Data FFS & Other Opps", startrow = 1)
    writer.save()
    writer.close()   

In [230]:
def edit_workbook_MD6(file):
    book = load_workbook(file)
    sheet = book["SF Data FFS & Other Opps"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Account Name"    
    sheet['C2']= "Opportunity Name"
    sheet['D2']= "Relationship Manager" 
    sheet['E2']= "Stage"
    sheet['F2']= "Close Date"    
    sheet['G2']= "Probability (%)"   
    sheet['H2']= "Amount"  
    sheet['I2']= "Weighted Value of Opportunity"   
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    sheet['D2'].font = Font(bold=True) 
    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:I1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['I1']
    top_left_cell.value = "SalesForce FY20+ Data as of " +timestamptoday 
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="DDDDDD")
    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(b=True, color="FFFFFF")
    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 25
    sheet.column_dimensions['D'].width = 25
    sheet.column_dimensions['E'].width = 25
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 25
    sheet.column_dimensions['H'].width = 25
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(7, 10):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '#,##0;[Red](#,##0)'
    for col in range(1, 6):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,10):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(9, 10):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(top=thin, left=thin, right=thin, bottom=thin)
    for col in range(6, 10):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).font = Font(bold=False)
    sheet.sheet_view.zoomScale = 70  
    sheet.freeze_panes = sheet['F3']
    book.save(file)

In [231]:
def format_multi_pivot_MD6(Group, str):
    step1 = create_MD_pivot6(Group)
    to_excel_MD6(step1, MDpath+str)
    edit_workbook_MD6(MDpath+str)

In [232]:
format_multi_pivot_MD6(Developing_Economies_RollupSFffs, '\\Developing Economies - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(ClimateAligned_IndustriesSFffs, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(IndiaSFffs, '\\India - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD6(OperationsSFffs, '\\Operations - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(China_RollupSFffs, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(Breakthrough_Technology_SFffs, '\\Breakthrough Technology - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(CarbonFree_MobilitySFffs, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(Carbon_Free_ElectricitySFffs, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(Strategic_Engagement_and_Analysis_GroupSFffs, '\\Strategic Engagement and Analysis Group - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(CarbonFreeBuildings_RollupSFffs, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(UrbanTrans_SFffs, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(ClimateIntelSFffs, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(IndustrySFffs, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(USSFffs, '\\US - '+timestampMD+' - MD Package.xlsx')
